### 다음 데이터 크롤링 (2021-03-07)
#### 웹툰정보 크롤링
* 평점순 웹툰 : http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=9999&page_no=1
* 웹툰 이름, 장르, 작가, 게시일, url, 줄거리, 평점, 19금여부, 완결여부 크롤링

In [ ]:
from IPython.display import clear_output # clear_output() 으로 아웃풋 제거 가능
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk
#notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해준다.
%matplotlib inline 
%config InlineBackend.figure_formats = {'png','retina'} #그래프를 더 높은 해상도를 출력

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [ ]:
#scrapy
!pip install scrapy

In [ ]:
import scrapy
import requests
from scrapy.http import TextResponse
import time
import urllib
import urllib3

In [ ]:
from pandas.io.json import json_normalize

# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
def get_data(url):
    response = requests.get(url)
    json_info = response.json() #json -> dict 형식으로 알아서 바꿔줌
    
    return json_info

In [ ]:
#19금 만화도 크롤링하려면 로그인을 해야한다

import getpass

myid = getpass.getpass('input your id : ')
mypassword = getpass.getpass('input your password : ')

LOGIN_URL = 'https://logins.daum.net/accounts/loginform.do'
LOGIN_DATA = {
    'id': myid,
    'inputPwd': mypassword
}
with requests.Session() as s:
    res = s.post(LOGIN_URL, data=LOGIN_DATA, verify=False, allow_redirects=False)

    print(res.status_code)

In [ ]:
# 평점순 웹툰url
# url = 'http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=50&page_no=1&1615108397290'

# -> page_size=9999(9999번까지) & page_no=1(1페이지로)
url1 = 'http://m.webtoon.daum.net/data/mobile/webtoon?sort=score&page_size=9999&page_no=1'
wbtn_raw1 = get_data(url1)
# wbtn_raw1

In [ ]:
wbt1_df = pd.DataFrame(columns = ['id', 'nickname', 'webtoonType', 'title', 'englishTitle', 'finishYn', 'titleImage2', 'thumbnailImage', 'thumbnailImage2', 'padThumbnailImage', 'artistCommentImage', 'homeThumbnailImage', 'appThumbnailImage', 'pcThumbnailImage', 'pcRecommendImage', 'pcHomeImage', 'pcHomeRightImage', 'pcHomeLeftColor', 'pcHomeRightColor', 'introduction', 'startDate', 'finishDate', 'restartDate', 'serviceStatus', 'viewerType', 'weekTerm', 'articleId', 'media', 'url', 'simpleUrl', 'webtoonGroupId', 'payYn', 'payType', 'price', 'ageGrade', 'restYn', 'monopolize', 'dateCreated', 'webtoonComment', 'cartoon', 'cp', 'webtoonWeeks', 'webtoonEpisodes', 'previewWebtoonEpisodes', 'latestWebtoonEpisode', 'webtoonServices', 'relatedProducts', 'promotionContents', 'score', 'tag', 'isNew', 'averageScore', 'seriesYn', 'ranking', 'diff', 'metricsScore', 'eventBanners', 'sort', 'sortWeight', 'gidamoo', 'webtoonNotice', 'couponAvailable'])

for data in wbtn_raw1['data']['webtoons']:
    wbt1_df.loc[len(wbt1_df)] = data

# wbt1_df.head(20)

In [ ]:
# # NaN,None값으로만 이루어진 column
# nulldf = wbt1_df.isnull().sum()>=603
# nulldf[nulldf==False].keys()

한 개의 column선택
* df["account"]

1개 이상의 column 선택
* df[["account", "street", "state"]]

In [ ]:
# chkdf = wbt1_df['isNew']==False
# len(chkdf[chkdf==True].keys())

In [ ]:
wbt1_df2 = wbt1_df[['id', 'nickname', #웹툰 닉네임
                    'title', 'finishYn', 'thumbnailImage2', #'appThumbnailImage', 
                    #'viewerType', #(1206개중 1172개가 scroll) 'articleId', 'price' (1206개 다 0), 
                    'ageGrade', # (1206개중 1004개 0) #'restYn', # (1206개중 1153개가 'N') -> 휴재 여부인 듯
                    'dateCreated', 'cartoon', 'latestWebtoonEpisode', 'webtoonServices', 'averageScore'
                    # 'score',(1206개 다 0) #'isNew', # 신작 여부인 듯 'ranking', 'diff', 'metricsScore', 'sortWeight', 'couponAvailable'
        ]]
# wbt1_df2

In [ ]:
"""
'ageGrade': 0,
    'appThumbnailImage': {'encryptKey': None,
     'height': 0,
     'id': 0,
     'mediaType': None,
     'name': None,
     'serviceStatus': 'service',
     'size': None,
     'url': 'http://t1.daumcdn.net/webtoon/op/937d2b06e473f16a750f176c749473de67735eb1',
     'width': 0},
    'articleId': 0,
    'artistCommentImage': None,
    'averageScore': 9.957233201581028,
    'cartoon': {'artists': [{'agency': None,
       'appImage': None,
       'artistOrder': 0,
       'artistType': None,
       'authUserinfo': None,
       'birthDay': None,
       'blog': None,
       'breakDay': None,
       'cafe': None,
       'career': None,
       'daumEmail': None,
       'daumEmailDisplayYn': None,
       'debutDay': None,
       'email': None,
       'facebook': None,
       'fancafe': None,
       'history': None,
       'homepage': None,
       'id': 0,
       'introduction': None,
       'joinDay': None,
       'name': '심우도',
       'nameImage': None,
       'penName': '심우도',
       'pictureImage': None,
       'smallPictureImage': None,
       'team': None,
       'teamYn': None,
       'twitter': None},
      {'agency': None,
       'appImage': None,
       'artistOrder': 0,
       'artistType': None,
       'authUserinfo': None,
       'birthDay': None,
       'blog': None,
       'breakDay': None,
       'cafe': None,
       'career': None,
       'daumEmail': None,
       'daumEmailDisplayYn': None,
       'debutDay': None,
       'email': None,
       'facebook': None,
       'fancafe': None,
       'history': None,
       'homepage': None,
       'id': 0,
       'introduction': None,
       'joinDay': None,
       'name': '심우도',
       'nameImage': None,
       'penName': '심우도',
       'pictureImage': None,
       'smallPictureImage': None,
       'team': None,
       'teamYn': None,
       'twitter': None}],
     'categories': None,
     'englishTitle': None,
     'genres': None,
     'id': 0,
     'introduction': None,
     'joinArtists': None,
     'searchId': None,
     'title': None,
     'titleImage': None},
    'couponAvailable': False,
    'cp': None,
    'dateCreated': '20180322143808',
    'diff': 0,
    'englishTitle': None,
    'eventBanners': None,
    'finishDate': None,
    'finishYn': 'Y',
    'gidamoo': {'dateCreated': '20200211142628',
     'endDate': None,
     'ended': False,
     'excludeEpisodes': 0,
     'id': 401,
     'interval': 48,
     'period': 3,
     'webtoonId': 1444},
    'homeThumbnailImage': None,
    'id': 1444,
    'introduction': None,
    'isNew': False,
    'latestWebtoonEpisode': {'ageGrade': 0,
     'articleId': 0,
     'commentUseYn': None,
     'dateCreated': '20190430000000',
     'encryptUseYn': None,
     'episode': 44,
     'episodeImage': None,
     'id': 66854,
     'isPaid': False,
     'isTopRecommend': False,
     'multiBackgroundImage': None,
     'multiBgm': None,
     'multiType': None,
     'padtoonImage': None,
     'payWebtoonEpisode': None,
     'previewEndDate': None,
     'price': 0,
     'product': None,
     'serviceStatus': None,
     'serviceType': None,
     'shareVoteTarget': None,
     'shortTitle': None,
     'simpleUrl': None,
     'specialSearchString': None,
     'specialSearchUrl': None,
     'thumbnailImage': {'encryptKey': None,
      'height': 0,
      'id': 66854,
      'mediaType': None,
      'name': None,
      'serviceStatus': 'service',
      'size': None,
      'url': 'http://t1.daumcdn.net/webtoon/op/96109f34fb088c0148fff2ac3198d664a9331b3a',
      'width': 0},
     'ticketAvailable': False,
     'title': '연재 후기',
     'voteTarget': None,
     'webtoon': None},
    'media': None,
    'metricsScore': 0,
    'monopolize': None,
    'nickname': 'vacantly',
    'padThumbnailImage': None,
    'payType': None,
    'payYn': None,
    'pcHomeImage': None,
    'pcHomeLeftColor': None,
    'pcHomeRightColor': None,
    'pcHomeRightImage': None,
    'pcRecommendImage': None,
    'pcThumbnailImage': None,
    'previewWebtoonEpisodes': None,
    'price': 0,
    'promotionContents': None,
    'ranking': 0,
    'relatedProducts': None,
    'restYn': 'N',
    'restartDate': None,
    'score': 0.0,
    'seriesYn': None,
    'serviceStatus': None,
    'simpleUrl': None,
    'sort': None,
    'sortWeight': 0,
    'startDate': None,
    'tag': None,
    'thumbnailImage': None,
    'thumbnailImage2': {'encryptKey': None,
     'height': 0,
     'id': 0,
     'mediaType': None,
     'name': None,
     'serviceStatus': 'service',
     'size': None,
     'url': 'http://t1.daumcdn.net/webtoon/op/22a74c4f0490c8883cb1b253ae271ef38df0851a',
     'width': 0},
    'title': '우두커니',
    'titleImage2': None,
    'url': None,
    'viewerType': 'scroll',
    'webtoonComment': None,
    'webtoonEpisodes': None,
    'webtoonGroupId': None,
    'webtoonNotice': None,
    'webtoonServices': [{'id': 35226,
      'serviceTarget': 'series',
      'webtoonId': 1444},
     {'id': 35227, 'serviceTarget': 'padtoon', 'webtoonId': 1444},
     {'id': 35228, 'serviceTarget': 'mobile', 'webtoonId': 1444},
     {'id': 35229, 'serviceTarget': 'app', 'webtoonId': 1444}],
    'webtoonType': None,
    'webtoonWeeks': None,
    'weekTerm': None},
    """

In [ ]:
# 썸네일 이미지 링크 정제

new_temp = pd.DataFrame(columns=['url'])
for data in wbt1_df2['thumbnailImage2']:
    new_temp.loc[len(new_temp)] = data['url']
# new_temp.tail()

In [ ]:
# 글/그림 작가 모으기

new_temp2 = pd.DataFrame(columns=['artist_name'])
for data in wbt1_df2['cartoon']:
    new_temp2.loc[len(new_temp2)] = data['artists'][0]['name']+'/'+data['artists'][1]['name']
# new_temp2

In [ ]:
# 마지막 업로드 일자/ 현재(완료) 회차

new_temp3 = pd.DataFrame(columns=['count_chapter'])
new_temp4 = pd.DataFrame(columns=['latest_Created'])
for data in wbt1_df2['latestWebtoonEpisode']:
    try:
        new_temp3.loc[len(new_temp3)]= data['episode']
    except:
        new_temp3.loc[len(new_temp3)]= None
    try:
        new_temp4.loc[len(new_temp4)]= data['dateCreated']
    except:
        new_temp4.loc[len(new_temp4)]= None     
# new_temp3
# new_temp4

In [ ]:
df = wbt1_df2[['id', 'nickname', 'title', 'finishYn', 'ageGrade', 'dateCreated', 'averageScore']]
df['artist']=new_temp2
df['latest_upload']=new_temp4
df['last_chapter']=new_temp3
df['img_url']=new_temp
# df

In [ ]:
# 장르/카테고리, 소개글, 자동수집

intro = []
genres = []
categories = []
print('trying to getting datas from Daum Webtoon...', end='')
for i, data in df.iterrows():
    url2 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+data['nickname']
    temp_df= get_data(url2)
    try:
        intro.append(temp_df['data']['webtoon']['introduction'])
    except:
        intro.append(None)
    gs=[]
    try:
        for data in temp_df['data']['webtoon']['cartoon']['genres']:
            gs.append(data['name'])
        gs_text = '/'.join(gs)
        genres.append(gs_text)
    except:
        genres.append(None)
    ls = []
    try:
        for data in temp_df['data']['webtoon']['cartoon']['categories']:
            ls.append(data['name'])
        tmp_text = '/'.join(ls)
        categories.append(tmp_text)
    except:
        categories.append(None)
    print(i,".", end='')
print("Crawling complete!")

df['intro'] = pd.Series(intro)
df['genres'] = pd.Series(genres)
df['categories'] = pd.Series(categories)
# df

trying to getting datas from Daum Webtoon...0 .1 .2 .3 .4 .5 .6 .7 .8 .9 .10 .11 .12 .13 .14 .15 .16 .17 .18 .19 .20 .21 .22 .23 .24 .25 .26 .27 .28 .29 .30 .31 .32 .33 .34 .35 .36 .37 .38 .39 .40 .41 .42 .43 .44 .45 .46 .47 .48 .49 .50 .51 .52 .53 .54 .55 .56 .57 .58 .59 .60 .61 .62 .63 .64 .65 .66 .67 .68 .69 .70 .71 .72 .73 .74 .75 .76 .77 .78 .79 .80 .81 .82 .83 .84 .85 .86 .87 .88 .89 .90 .91 .92 .93 .94 .95 .96 .97 .98 .99 .100 .101 .102 .103 .104 .105 .106 .107 .108 .109 .110 .111 .112 .113 .114 .115 .116 .117 .118 .119 .120 .121 .122 .123 .124 .125 .126 .127 .128 .129 .130 .131 .132 .133 .134 .135 .136 .137 .138 .139 .140 .141 .142 .143 .144 .145 .146 .147 .148 .149 .150 .151 .152 .153 .154 .155 .156 .157 .158 .159 .160 .161 .162 .163 .164 .165 .166 .167 .168 .169 .170 .171 .172 .173 .174 .175 .176 .177 .178 .179 .180 .181 .182 .183 .184 .185 .186 .187 .188 .189 .190 .191 .192 .193 .194 .195 .196 .197 .198 .199 .200 .201 .202 .203 .204 .205 .206 .207 .208 .209 .210 .211 .212 .2

In [ ]:
# 웹툰 url

w_url = []
for i, data in df.iterrows():
    w_url.append('http://webtoon.daum.net/webtoon/view/'+data['nickname'])

df['page url'] = pd.Series(w_url)
# df

In [ ]:
df.to_csv("D_Webtoon_1_toonsinfo.csv",index=False)